In [14]:

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
import numpy as np
from mlflow.models.signature import infer_signature
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [15]:
load_dotenv()

access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_url = os.getenv("MLFLOW_S3_ENDPOINT_URL")
tracker_url = os.getenv("MLFLOW_URL")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_ADMIN_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_ADMIN_PASSWORD")
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
bucket_name = "dataset"
object_name = "Iris.csv"

In [16]:
print(s3_url)
print(tracker_url)
print(os.getenv("MLFLOW_TRACKING_PASSWORD"))

https://mlops-s3.fauzanghaza.com
https://mlops-mlflow.fauzanghaza.com
irul12345


In [17]:
s3 = boto3.client(
    's3',
    endpoint_url=s3_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    verify=False
)

In [18]:
try:
    # Attempt to list buckets
    response = s3.list_buckets()
    
    # If successful, print bucket names
    print("Connected successfully! Buckets available:")
    for bucket in response['Buckets']:
        print(f"- {bucket['Name']}")
except Exception as e:
    # Print any connection errors
    print("Connection error:", e)


Connected successfully! Buckets available:
- dataset
- mlflow


In [19]:
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    print(response)
    dataset_content = response.get('Body')
    print(dataset_content)
    # Load the dataset into a DataFrame
    df = pd.read_csv(dataset_content)
    print("Dataset loaded successfully:")
    print(df.head())
except Exception as e:
    print("Error fetching dataset from MinIO:", e)


{'ResponseMetadata': {'RequestId': '180E0ACCD591136F', 'HostId': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 04 Dec 2024 17:53:01 GMT', 'content-type': 'text/csv', 'content-length': '5107', 'connection': 'keep-alive', 'etag': '"717820ef0af287ff346c5cabfb4c612c"', 'last-modified': 'Wed, 04 Dec 2024 17:42:48 GMT', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'b675f5c02385af78c69266a96b22038bea3ecd7423814977a7267fd68ac2bbeb', 'x-amz-request-id': '180E0ACCD591136F', 'x-amz-version-id': '1c6161e4-3c64-4f98-9491-965a4fc42235', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '2155', 'x-ratelimit-remaining': '2155', 'x-xss-protection': '1; mode=block', 'cf-cache-status': 'BYPASS', 'accept-ranges': 'bytes', 'report-to': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=Jf12ymQyi4OLvNWjcgV2E%2BLCsETBTX7OR3p6YF7gPGkLb9

In [20]:
mlflow.set_tracking_uri(tracker_url)
experiment_name = "test"
mlflow.set_experiment(experiment_name)

data = df.drop('Species', axis=1)
target = df['Species']
label_mapping = {
    'Iris-setosa': 0,
    'Iris-versicolor': 1,
    'Iris-virginica': 2
}
target = [label_mapping[label] for label in target]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [21]:
from mlflow.models.signature import infer_signature
import mlflow.xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.xgboost.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2024/12/05 00:53:03 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/12/05 00:53:03 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missin

Run ID: 7955edcd4c4744708be0df54bc7bbbee
Model accuracy: 1.0
🏃 View run unique-steed-641 at: https://mlops-mlflow.fauzanghaza.com/#/experiments/1/runs/7955edcd4c4744708be0df54bc7bbbee
🧪 View experiment at: https://mlops-mlflow.fauzanghaza.com/#/experiments/1


2024/12/05 00:53:10 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/05 00:53:11 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [22]:
import mlflow.pyfunc

model_uri = f"runs:/{run_id}/model"
model = mlflow.pyfunc.load_model(model_uri)

# Check if metadata and signature are available
if model.metadata:
    print("Model signature:", model.metadata.get_input_schema())
else:
    print("Model metadata is missing.")

Model signature: ['Id': long (required), 'SepalLengthCm': double (required), 'SepalWidthCm': double (required), 'PetalLengthCm': double (required), 'PetalWidthCm': double (required)]


In [23]:
from mlflow.models.signature import infer_signature
import mlflow.xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


2024/12/05 00:53:12 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/12/05 00:53:12 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/fauzanghaza/Applications/miniconda3/envs/MLOps/lib/python3.10/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missin

Run ID: bddf86cf3e9e4e3aac04254fb21b3aab
Model accuracy: 1.0
🏃 View run unequaled-seal-498 at: https://mlops-mlflow.fauzanghaza.com/#/experiments/1/runs/bddf86cf3e9e4e3aac04254fb21b3aab
🧪 View experiment at: https://mlops-mlflow.fauzanghaza.com/#/experiments/1


2024/12/05 00:53:19 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/12/05 00:53:19 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
